# E-commerce Business Intelligence Analysis

A comprehensive exploratory data analysis of e-commerce sales data focusing on business metrics, performance trends, and strategic insights.

## Table of Contents
1. [Introduction & Business Objectives](#introduction)
2. [Configuration & Setup](#configuration)
3. [Data Loading & Validation](#data-loading)
4. [Data Dictionary](#data-dictionary)
5. [Revenue Performance Analysis](#revenue-analysis)
6. [Product Category Performance](#product-analysis)
7. [Geographic Performance Analysis](#geographic-analysis)
8. [Customer Experience Analysis](#customer-analysis)
9. [Executive Summary & Recommendations](#summary)

---

## 1. Introduction & Business Objectives {#introduction}

This analysis provides comprehensive business intelligence insights for e-commerce operations, focusing on key performance indicators and strategic metrics.

### Primary Business Questions:
- How has revenue performance changed over time?
- Which product categories drive the most revenue?
- What are the geographic patterns in sales performance?
- How satisfied are customers with our service?
- What delivery performance metrics indicate operational efficiency?

### Key Performance Indicators (KPIs):
- **Revenue Metrics**: Total revenue, growth rates, average order value
- **Product Performance**: Category revenue distribution, top performers
- **Geographic Insights**: State-level performance analysis
- **Customer Satisfaction**: Review scores and delivery experience
- **Operational Efficiency**: Delivery times and fulfillment rates

## 2. Configuration & Setup {#configuration}

Configure analysis parameters to customize the time period and comparison metrics.

In [ ]:
# Analysis Configuration Parameters
ANALYSIS_YEAR = 2023        # Primary year to analyze
COMPARISON_YEAR = 2022      # Comparison year for growth calculations
ANALYSIS_MONTH = None       # Specific month (1-12) or None for full year
DATA_PATH = 'ecommerce_data/'

# Display configuration
print(f"Analysis Period: {ANALYSIS_YEAR}")
print(f"Comparison Period: {COMPARISON_YEAR}")
print(f"Month Filter: {'Full Year' if ANALYSIS_MONTH is None else f'Month {ANALYSIS_MONTH}'}")
print(f"Data Source: {DATA_PATH}")

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Import custom modules
from data_loader import EcommerceDataLoader, load_and_process_data
from business_metrics import BusinessMetricsCalculator, MetricsVisualizer, print_metrics_summary

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

print("Libraries and modules loaded successfully")

## 3. Data Loading & Validation {#data-loading}

Load and validate all e-commerce datasets using the modular data loading framework.

In [ ]:
# Load and process all data using the data loader module
print("Loading e-commerce datasets...")
loader, processed_data = load_and_process_data(DATA_PATH)

# Display data summary
summary = loader.get_data_summary()
print("\nDataset Summary:")
print("=" * 50)
for dataset_name, info in summary.items():
    print(f"{dataset_name.upper()}:")
    print(f"  Rows: {info['rows']:,}")
    print(f"  Columns: {info['columns']}")
    print(f"  Memory Usage: {info['memory_usage_mb']:.1f} MB")
    if info['date_range']:
        print(f"  Date Range: {info['date_range']['start'].date()} to {info['date_range']['end'].date()}")
    print()

In [ ]:
# Create comprehensive sales dataset for analysis
print("Creating comprehensive sales dataset...")
sales_data = loader.create_sales_dataset(
    year_filter=None,  # Load all years for comparison
    month_filter=ANALYSIS_MONTH,
    status_filter='delivered'  # Focus on completed orders
)

print(f"Sales dataset created with {len(sales_data):,} records")
print(f"Date range: {sales_data['order_purchase_timestamp'].min().date()} to {sales_data['order_purchase_timestamp'].max().date()}")

# Display sample data
print("\nSample sales data:")
display(sales_data.head())

print(f"\nData quality check:")
print(f"Unique orders: {sales_data['order_id'].nunique():,}")
print(f"Unique products: {sales_data['product_id'].nunique():,}")
print(f"Unique customers: {sales_data['customer_id'].nunique():,}")
print(f"Average order value: ${sales_data.groupby('order_id')['price'].sum().mean():.2f}")

## 4. Data Dictionary {#data-dictionary}

Understanding key columns and business terms used throughout the analysis.

### Key Business Metrics

| Metric | Definition | Business Significance |
|--------|------------|----------------------|
| **Revenue** | Sum of delivered order item prices | Core business performance indicator |
| **Average Order Value (AOV)** | Total revenue divided by number of orders | Customer spending behavior metric |
| **Revenue Growth Rate** | Year-over-year percentage change in revenue | Business expansion indicator |
| **Customer Satisfaction** | Average review score (1-5 scale) | Service quality measurement |
| **Delivery Performance** | Average days from purchase to delivery | Operational efficiency indicator |

### Data Schema

| Column | Description | Data Type |
|--------|-------------|----------|
| `order_id` | Unique identifier for each order | String |
| `customer_id` | Unique identifier for each customer | String |
| `product_id` | Unique identifier for each product | String |
| `price` | Item price in USD | Float |
| `order_purchase_timestamp` | Date and time of order placement | Datetime |
| `order_status` | Current status of the order | String |
| `product_category_name` | Product category classification | String |
| `customer_state` | Customer's state location | String |
| `review_score` | Customer satisfaction rating (1-5) | Integer |
| `delivery_days` | Days from order to delivery | Integer |

### Order Status Categories
- **delivered**: Successfully completed orders (primary focus)
- **shipped**: Orders in transit
- **processing**: Orders being prepared
- **canceled**: Canceled orders
- **pending**: Orders awaiting processing
- **returned**: Returned orders

## 5. Revenue Performance Analysis {#revenue-analysis}

Comprehensive analysis of revenue trends, growth patterns, and key financial metrics.

In [ ]:
# Initialize business metrics calculator
metrics_calculator = BusinessMetricsCalculator(sales_data)

# Calculate comprehensive revenue metrics
print(f"Calculating revenue metrics for {ANALYSIS_YEAR}...")
revenue_metrics = metrics_calculator.calculate_revenue_metrics(
    current_year=ANALYSIS_YEAR,
    previous_year=COMPARISON_YEAR
)

# Display revenue performance summary
print("\nREVENUE PERFORMANCE SUMMARY")
print("=" * 40)
print(f"Analysis Period: {ANALYSIS_YEAR}")
print(f"Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Average Order Value: ${revenue_metrics['average_order_value']:.2f}")
print(f"Total Items Sold: {revenue_metrics['total_items_sold']:,}")

if COMPARISON_YEAR:
    print(f"\nCOMPARISON TO {COMPARISON_YEAR}:")
    print(f"Revenue Growth: {revenue_metrics['revenue_growth_rate']:.2f}%")
    print(f"Order Growth: {revenue_metrics['order_growth_rate']:.2f}%")
    print(f"AOV Growth: {revenue_metrics['aov_growth_rate']:.2f}%")

print("\n" + "="*40)

In [ ]:
# Monthly revenue trend analysis
monthly_trends = metrics_calculator.calculate_monthly_trends(ANALYSIS_YEAR)

print(f"Monthly Revenue Trends for {ANALYSIS_YEAR}:")
display(monthly_trends)

# Calculate average monthly growth
avg_monthly_growth = monthly_trends['revenue_growth'].mean()
print(f"\nAverage Monthly Growth Rate: {avg_monthly_growth:.2f}%")

In [ ]:
# Create revenue trend visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Monthly revenue trend
ax1.plot(monthly_trends['month'], monthly_trends['revenue'], 
         marker='o', linewidth=3, markersize=8, color='#1f77b4')
ax1.set_title(f'Monthly Revenue Trend - {ANALYSIS_YEAR}', fontsize=16, fontweight='bold', pad=20)
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Revenue ($)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add data labels
for i, v in enumerate(monthly_trends['revenue']):
    ax1.annotate(f'${v:,.0f}', (monthly_trends['month'].iloc[i], v), 
                textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)

# Monthly growth rate
colors = ['green' if x > 0 else 'red' for x in monthly_trends['revenue_growth'].dropna()]
ax2.bar(monthly_trends['month'].iloc[1:], monthly_trends['revenue_growth'].dropna(), color=colors, alpha=0.7)
ax2.set_title(f'Month-over-Month Revenue Growth Rate - {ANALYSIS_YEAR}', fontsize=16, fontweight='bold', pad=20)
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Growth Rate (%)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)

plt.tight_layout()
plt.show()

### Revenue Performance Insights

**Key Findings:**
- Revenue performance shows seasonal patterns with variations throughout the year
- Month-over-month growth rates indicate business volatility and trend directions
- Average order value trends reflect customer spending behavior changes

## 6. Product Category Performance {#product-analysis}

Analysis of product category revenue distribution, top performers, and market share insights.

In [ ]:
# Analyze product category performance
print(f"Analyzing product performance for {ANALYSIS_YEAR}...")
product_performance = metrics_calculator.analyze_product_performance(ANALYSIS_YEAR, top_n=13)

if 'error' not in product_performance:
    print("\nTOP 10 PRODUCT CATEGORIES BY REVENUE:")
    print("=" * 50)
    top_categories = product_performance['top_categories'].head(10)
    
    for idx, row in top_categories.iterrows():
        print(f"{row['product_category_name'].replace('_', ' ').title():}")
        print(f"  Revenue: ${row['total_revenue']:,.2f} ({row['revenue_share']:.1f}% of total)")
        print(f"  Items Sold: {row['items_sold']:,}")
        print(f"  Avg Item Price: ${row['avg_item_price']:.2f}")
        print(f"  Unique Orders: {row['unique_orders']:,}")
        print()
    
    # Display full category table
    print("COMPLETE CATEGORY PERFORMANCE:")
    display(product_performance['all_categories'])
else:
    print("Product category data not available")

In [ ]:
# Product category revenue visualization
if 'error' not in product_performance:
    top_10_categories = product_performance['top_categories'].head(10)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Horizontal bar chart for top categories
    colors = plt.cm.Blues(np.linspace(0.4, 0.8, len(top_10_categories)))
    bars = ax1.barh(range(len(top_10_categories)), top_10_categories['total_revenue'], color=colors)
    ax1.set_yticks(range(len(top_10_categories)))
    ax1.set_yticklabels([cat.replace('_', ' ').title() for cat in top_10_categories['product_category_name']])
    ax1.set_xlabel('Revenue ($)', fontsize=12)
    ax1.set_title(f'Top 10 Product Categories by Revenue - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # Add value labels
    for i, v in enumerate(top_10_categories['total_revenue']):
        ax1.text(v + max(top_10_categories['total_revenue']) * 0.01, i, f'${v:,.0f}', 
                va='center', fontsize=10)
    
    # Revenue share pie chart
    top_5_for_pie = top_10_categories.head(5)
    other_revenue = top_10_categories.iloc[5:]['total_revenue'].sum()
    
    pie_data = list(top_5_for_pie['total_revenue']) + [other_revenue]
    pie_labels = list(top_5_for_pie['product_category_name'].str.replace('_', ' ').str.title()) + ['Other Categories']
    
    wedges, texts, autotexts = ax2.pie(pie_data, labels=pie_labels, autopct='%1.1f%%', 
                                      colors=plt.cm.Set3(np.linspace(0, 1, len(pie_data))))
    ax2.set_title(f'Revenue Share by Top Categories - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot create product visualizations - category data not available")

### Product Category Insights

**Key Findings:**
- Product category performance reveals market segment strengths and opportunities
- Revenue concentration indicates dependency on top-performing categories
- Average item prices vary significantly across categories, indicating different market positioning

## 7. Geographic Performance Analysis {#geographic-analysis}

State-level analysis of sales performance, revenue distribution, and market penetration.

In [ ]:
# Analyze geographic performance
print(f"Analyzing geographic performance for {ANALYSIS_YEAR}...")
geographic_performance = metrics_calculator.analyze_geographic_performance(ANALYSIS_YEAR)

if 'error' not in geographic_performance.columns:
    print("\nTOP 15 STATES BY REVENUE:")
    print("=" * 40)
    top_states = geographic_performance.head(15)
    
    for idx, row in top_states.iterrows():
        print(f"{row['state']}:")
        print(f"  Revenue: ${row['revenue']:,.2f}")
        print(f"  Orders: {row['orders']:,}")
        print(f"  Avg Order Value: ${row['avg_order_value']:.2f}")
        print()
    
    # Geographic summary statistics
    print("GEOGRAPHIC PERFORMANCE SUMMARY:")
    print(f"Total States with Sales: {len(geographic_performance)}")
    print(f"Top State Revenue: ${geographic_performance.iloc[0]['revenue']:,.2f}")
    print(f"Median State Revenue: ${geographic_performance['revenue'].median():,.2f}")
    print(f"Revenue Concentration (Top 5 States): {(geographic_performance.head(5)['revenue'].sum() / geographic_performance['revenue'].sum() * 100):.1f}%")
else:
    print("Geographic data not available for analysis")

In [ ]:
# Geographic visualizations
if 'error' not in geographic_performance.columns:
    # Create choropleth map using Plotly
    fig_map = px.choropleth(
        geographic_performance,
        locations='state',
        color='revenue',
        locationmode='USA-states',
        scope='usa',
        title=f'Revenue by State - {ANALYSIS_YEAR}',
        color_continuous_scale='Blues',
        labels={'revenue': 'Revenue ($)'}
    )
    
    fig_map.update_layout(
        title_font_size=16,
        title_x=0.5,
        geo=dict(showframe=False, showcoastlines=True),
        width=1000,
        height=600
    )
    
    fig_map.show()
    
    # Top states bar chart
    top_10_states = geographic_performance.head(10)
    
    plt.figure(figsize=(15, 8))
    bars = plt.bar(top_10_states['state'], top_10_states['revenue'], 
                   color='skyblue', alpha=0.8)
    plt.title(f'Top 10 States by Revenue - {ANALYSIS_YEAR}', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('State', fontsize=12)
    plt.ylabel('Revenue ($)', fontsize=12)
    plt.xticks(rotation=45)
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    plt.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, top_10_states['revenue']):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(top_10_states['revenue']) * 0.01,
                f'${value:,.0f}', ha='center', va='bottom', fontsize=10, rotation=0)
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot create geographic visualizations - location data not available")

### Geographic Performance Insights

**Key Findings:**
- Geographic revenue distribution reveals market concentration and expansion opportunities
- State-level performance variations indicate regional preferences and market maturity
- Average order values by state provide insights into regional customer spending patterns

## 8. Customer Experience Analysis {#customer-analysis}

Comprehensive analysis of customer satisfaction, delivery performance, and service quality metrics.

In [ ]:
# Customer satisfaction analysis
print(f"Analyzing customer satisfaction for {ANALYSIS_YEAR}...")
satisfaction_metrics = metrics_calculator.analyze_customer_satisfaction(ANALYSIS_YEAR)

if 'error' not in satisfaction_metrics:
    print("\nCUSTOMER SATISFACTION METRICS:")
    print("=" * 40)
    print(f"Average Review Score: {satisfaction_metrics['avg_review_score']:.2f}/5.0")
    print(f"Total Reviews: {satisfaction_metrics['total_reviews']:,}")
    print(f"5-Star Reviews: {satisfaction_metrics['score_5_percentage']:.1f}%")
    print(f"4+ Star Reviews: {satisfaction_metrics['score_4_plus_percentage']:.1f}%")
    print(f"Low Satisfaction (1-2 Stars): {satisfaction_metrics['score_1_2_percentage']:.1f}%")
else:
    print("Customer satisfaction data not available")

# Delivery performance analysis
print(f"\nAnalyzing delivery performance for {ANALYSIS_YEAR}...")
delivery_metrics = metrics_calculator.analyze_delivery_performance(ANALYSIS_YEAR)

if 'error' not in delivery_metrics:
    print("\nDELIVERY PERFORMANCE METRICS:")
    print("=" * 40)
    print(f"Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
    print(f"Median Delivery Time: {delivery_metrics['median_delivery_days']:.1f} days")
    print(f"Fast Delivery (≤3 days): {delivery_metrics['fast_delivery_percentage']:.1f}%")
    print(f"Slow Delivery (>7 days): {delivery_metrics['slow_delivery_percentage']:.1f}%")
else:
    print("Delivery performance data not available")

In [ ]:
# Create customer experience visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Review score distribution
if 'error' not in satisfaction_metrics:
    # Get review distribution from sales data
    current_year_data = sales_data[sales_data['purchase_year'] == ANALYSIS_YEAR]
    review_counts = current_year_data.drop_duplicates('order_id')['review_score'].value_counts().sort_index()
    
    colors = ['#d62728', '#ff7f0e', '#ffbb78', '#2ca02c', '#1f77b4']
    bars = axes[0,0].bar(review_counts.index, review_counts.values, color=colors)
    axes[0,0].set_title(f'Review Score Distribution - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
    axes[0,0].set_xlabel('Review Score', fontsize=12)
    axes[0,0].set_ylabel('Number of Reviews', fontsize=12)
    axes[0,0].grid(True, alpha=0.3)
    
    # Add percentage labels
    total_reviews = review_counts.sum()
    for bar, count in zip(bars, review_counts.values):
        percentage = count / total_reviews * 100
        axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + total_reviews * 0.01,
                      f'{percentage:.1f}%', ha='center', va='bottom', fontsize=10)
else:
    axes[0,0].text(0.5, 0.5, 'Review data not available', ha='center', va='center', 
                  transform=axes[0,0].transAxes, fontsize=14)

# Delivery time distribution
if 'error' not in delivery_metrics:
    delivery_data = current_year_data.drop_duplicates('order_id')['delivery_days'].dropna()
    
    axes[0,1].hist(delivery_data, bins=30, color='skyblue', alpha=0.7, edgecolor='black')
    axes[0,1].axvline(delivery_data.mean(), color='red', linestyle='--', linewidth=2, 
                     label=f'Mean: {delivery_data.mean():.1f} days')
    axes[0,1].axvline(delivery_data.median(), color='green', linestyle='--', linewidth=2,
                     label=f'Median: {delivery_data.median():.1f} days')
    axes[0,1].set_title(f'Delivery Time Distribution - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
    axes[0,1].set_xlabel('Delivery Days', fontsize=12)
    axes[0,1].set_ylabel('Number of Orders', fontsize=12)
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
else:
    axes[0,1].text(0.5, 0.5, 'Delivery data not available', ha='center', va='center', 
                  transform=axes[0,1].transAxes, fontsize=14)

# Satisfaction by delivery speed
if 'error' not in satisfaction_metrics and 'error' not in delivery_metrics:
    # Categorize delivery times
    order_data = current_year_data.drop_duplicates('order_id').copy()
    order_data['delivery_category'] = pd.cut(
        order_data['delivery_days'], 
        bins=[0, 3, 7, float('inf')], 
        labels=['1-3 days', '4-7 days', '8+ days']
    )
    
    satisfaction_by_speed = order_data.groupby('delivery_category')['review_score'].mean()
    
    bars = axes[1,0].bar(range(len(satisfaction_by_speed)), satisfaction_by_speed.values, 
                        color=['#2ca02c', '#ff7f0e', '#d62728'])
    axes[1,0].set_xticks(range(len(satisfaction_by_speed)))
    axes[1,0].set_xticklabels(satisfaction_by_speed.index)
    axes[1,0].set_title(f'Average Review Score by Delivery Speed - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
    axes[1,0].set_xlabel('Delivery Time Category', fontsize=12)
    axes[1,0].set_ylabel('Average Review Score', fontsize=12)
    axes[1,0].set_ylim(0, 5)
    axes[1,0].grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, satisfaction_by_speed.values):
        axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                      f'{value:.2f}', ha='center', va='bottom', fontsize=11)
else:
    axes[1,0].text(0.5, 0.5, 'Insufficient data for analysis', ha='center', va='center', 
                  transform=axes[1,0].transAxes, fontsize=14)

# Order status distribution
status_data = sales_data[sales_data['purchase_year'] == ANALYSIS_YEAR]['order_status'].value_counts()
wedges, texts, autotexts = axes[1,1].pie(status_data.values, labels=status_data.index, autopct='%1.1f%%',
                                         colors=plt.cm.Set3(np.linspace(0, 1, len(status_data))))
axes[1,1].set_title(f'Order Status Distribution - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### Customer Experience Insights

**Key Findings:**
- Customer satisfaction levels indicate overall service quality and areas for improvement
- Delivery performance directly correlates with customer satisfaction scores
- Fast delivery (1-3 days) typically results in higher customer satisfaction ratings
- Order fulfillment rates demonstrate operational efficiency

## 9. Executive Summary & Recommendations {#summary}

Strategic insights and actionable recommendations based on comprehensive business intelligence analysis.

In [ ]:
# Generate comprehensive business report
print("Generating comprehensive business intelligence report...")
comprehensive_report = metrics_calculator.generate_comprehensive_report(
    current_year=ANALYSIS_YEAR,
    previous_year=COMPARISON_YEAR
)

# Print formatted summary
print_metrics_summary(comprehensive_report)

### Strategic Recommendations

Based on the comprehensive analysis, here are key strategic recommendations:

#### Revenue Optimization
- **Focus on High-Performing Categories**: Invest marketing resources in top revenue-generating product categories
- **Average Order Value Enhancement**: Implement cross-selling and upselling strategies to increase AOV
- **Seasonal Planning**: Develop targeted campaigns for months showing consistent growth patterns

#### Geographic Expansion
- **Market Penetration**: Focus on underperforming states with high potential
- **Regional Customization**: Tailor product offerings based on regional preferences and performance
- **Logistics Optimization**: Improve delivery networks in high-revenue states

#### Customer Experience Enhancement
- **Delivery Speed Improvement**: Prioritize faster delivery options to boost customer satisfaction
- **Quality Control**: Address factors contributing to low review scores (1-2 stars)
- **Customer Retention**: Develop loyalty programs for high-satisfaction customers

#### Operational Efficiency
- **Fulfillment Optimization**: Reduce delivery times, especially for orders taking more than 7 days
- **Inventory Management**: Ensure adequate stock for high-performing product categories
- **Process Improvement**: Streamline order processing to reduce cancellation rates

### Key Performance Indicators (KPIs) to Monitor

1. **Revenue Growth Rate**: Target consistent month-over-month growth
2. **Customer Satisfaction Score**: Maintain average review score above 4.0
3. **Delivery Performance**: Achieve >30% of orders delivered within 3 days
4. **Order Fulfillment Rate**: Maintain >95% successful delivery rate
5. **Geographic Diversification**: Reduce revenue concentration risk across states

### Next Steps and Action Items

1. **Data Collection Enhancement**
   - Implement more granular tracking for customer journey analysis
   - Add product profitability data for comprehensive category analysis
   - Include customer acquisition costs for ROI calculations

2. **Analysis Automation**
   - Schedule regular execution of this analysis for ongoing monitoring
   - Set up alerting for significant performance changes
   - Create executive dashboards for real-time KPI tracking

3. **Advanced Analytics**
   - Implement predictive models for demand forecasting
   - Develop customer segmentation for targeted marketing
   - Create churn prediction models for retention strategies

---

**Analysis Configuration Summary:**
- **Primary Analysis Period:** {ANALYSIS_YEAR}
- **Comparison Period:** {COMPARISON_YEAR}
- **Data Source:** {DATA_PATH}
- **Analysis Scope:** {'Full Year' if ANALYSIS_MONTH is None else f'Month {ANALYSIS_MONTH}'}

*This analysis was generated using a configurable framework that can be easily adapted for different time periods and business requirements.*